In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importamos 
Las librerias necesarias para hacer los modelos que utilizaremos luego, ademas de las métricas para evaluar los resultados.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error

from sklearn import tree
import matplotlib.pyplot as plt

# Input
Abrimos el dataset con la información suministrada por kaggle

In [ ]:
data = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv');
print(data.head())

Primero definimos categorias para armar el input después

In [ ]:
CATEGORIES = {
    'class': ['e', 'p'],
    'cap-shape': ['x', 'b', 'c', 'k', 's', 'f'], 
    'cap-surface': ['f', 'g', 'y', 's'], 
    'cap-color': ['n', 'b', 'c', 'g','r', 'p', 'u', 'e', 'w', 'y'], 
    'bruises': ['t', 'f'], 
    'odor': ['a', 'l', 'c', 'y', 'f', 'm', 'n', 'p', 's'],
    'gill-attachment': ['a', 'd', 'f', 'n'], 
    'gill-spacing': ['c', 'w', 'd'], 
    'gill-size': ['b', 'n'],
    'gill-color': ['k', 'n', 'b', 'h', 'g', 'r', 'o', 'p', 'u', 'e', 'w', 'y'],
    'stalk-shape': ['e', 't'], 
    'stalk-root': ['b', 'c', 'u', 'e', 'z', 'r', '?'], 
    'stalk-surface-above-ring': ['f', 'y', 'k', 's'],
    'stalk-surface-below-ring': ['f', 'y', 'k', 's'],
    'stalk-color-above-ring': ['b', 'n', 'c', 'g', 'o', 'p', 'e', 'w', 'y'],
    'stalk-color-below-ring': ['b', 'n', 'c', 'g', 'o', 'p', 'e', 'w', 'y'],
    'veil-type': ['p', 'u'], 
    'veil-color': ['n', 'o', 'w', 'y'], 
    'ring-number': ['o', 'n', 't'],
    'ring-type': ['c', 'e', 'f', 'l', 'n', 'p', 's', 'z'], 
    'spore-print-color': ['k', 'n', 'b', 'h', 'r', 'o', 'u', 'w', 'y'], 
    'population': ['a', 'c', 'n', 's', 'v', 'y'], 
    'habitat': ['g', 'l', 'm', 'p', 'u', 'w', 'd']  
}

Convertimos los datos de *string* a *enteros* segun los posibles valores que pueden tomar

In [ ]:
    
df = data
for feature, vocab in CATEGORIES.items():
    df[feature] = df[feature].apply(lambda x: vocab.index(x))
df.head()

Separamos los datos por un lado en test y train y por el otro en datos y labels (X e y)

In [ ]:
X = df.iloc[:, 1:23].values
y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)
X_train.shape

# Red Neuronal

Creamos el modelo que consistirá de el input (convertido a numeros anteriormente), una gran capa densa de 128 neuronas y una capa de salida con dos neuronas que corresponderan a cada una de las clases que se puede asignar a los hongos.

Esta red es la primera planteada y no fue necesaria cambiarla (como suele ser el caso) para que se ajuste mejor al dataset. Se utiliza relu como función de activación y 20 ciclos de entrenamiento (de los cuales en realidad solo eran necesarios 10 aproximadamente).

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(22,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=20)

Ahora generamos las metricas con ayuda de sklearn dado que la libreria de Keras no tiene soporte nativo para el F1 score

In [ ]:
y_predict = model.predict(X_test).argmax(axis=-1)
metrics = precision_recall_fscore_support(y_test, y_predict, average='binary')
print('Precision :', metrics[0])
print('Recall :', metrics[1])
print('Medida F1 :', metrics[2])
print('Error cuadratico medio: ', mean_squared_error(y_test, y_predict))


# Random Forrest
Ahora utilizando la misma información usamos el clasificador random forrest, como comentario importante hay que destacar la simplicidad de este clasificador no solo para hacer el codigo sino tambien para elegir hyperparametros.
También se eligió un unico arbol como estimador dada la simplicidad del problema (y veremos que no hacen falta más).

In [ ]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=2, random_state=0, n_estimators=1)
clf.fit(X_train, y_train)

In [ ]:
indice_tree = 0 # Which tree we want to plot
fig, axes = plt.subplots(figsize = (20,20), dpi=800)
tree.plot_tree(clf.estimators_[indice_tree],
               feature_names = list(CATEGORIES.keys()), 
               class_names=['edible', 'poisonous'],
               filled = True,
               rounded = True);

In [ ]:
y_forrest_predict = clf.predict(X_test)
metrics = precision_recall_fscore_support(y_test, y_forrest_predict, average='binary')
print('Precision :', metrics[0])
print('Recall :', metrics[1])
print('Medida F1 :', metrics[2])
print('Error cuadratico medio: ', mean_squared_error(y_test, y_forrest_predict))

# Conclusiones

Se puede observar que las predicciones dieron 100% correctas al igual que el clasificador de la red neuronal. En general veriamos resultados mejores en este ultimo clasificador pero debido a la facilidad del dataset no fue el caso. Esta facilidad viene dada no solo en cuanto a la predictibilidad sino tambien la limpieza del dataset y su muy buena documentación que tampoco suele ser el caso mas normal.